<a href="https://colab.research.google.com/github/cyclic-reference/fast-ai-swift/blob/master/initial_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmp9i4h4wuf/swift-install
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/saeta/Just
Fetching https://github.com/latenitesoft/NotebookExport
Completed resolution in 0.91s
Cloning https://github.com/latenitesoft/NotebookExport
Resolving https://github.com/latenitesoft/NotebookExport at 0.6.0
Cloning https://github.com/saeta/Just
Resolving https://github.com/saeta/Just at 0.7.3
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.3
[1/11] Compiling Path Path+StringConvertibles.swift
[2/11] Compiling Path Path+ls.swift
[3/11] Compiling Path Path+CommonDirectories.swift
[4/11] Compiling Path Path+FileManager.swift
[5/11] Comp

In [2]:
// export
precedencegroup ExponentiationPrecedence {
    associativity: right
    higherThan: MultiplicationPrecedence
}
infix operator ** : ExponentiationPrecedence

precedencegroup CompositionPrecedence { associativity: left }
infix operator >| : CompositionPrecedence

In [3]:
//export
import Foundation
import Just
import Path

In [4]:
//export
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String {
        let (task,pipe) = (Process(),Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments,task.standardOutput) = (args,pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

In [5]:
print("/bin/ls".shell("-lh"))

total 92K
-rw-rw-r-- 1 ubuntu ubuntu  460 Sep 14 22:52 blank_swift.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  12K Sep 14 22:52 convolutional_neural_network.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 4.0K Sep 15 12:41 deep-learning-box.sh
-rw-rw-r-- 1 ubuntu ubuntu  622 Sep 14 22:57 inference.swift
-rw-rw-r-- 1 ubuntu ubuntu  20K Sep 14 22:52 initial_setup.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 6.9K Sep 14 22:52 layers.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  33K Sep 14 22:52 matrix_multiplication.ipynb
drwxrwxr-x 4 ubuntu ubuntu 4.0K Sep 15 12:53 swift-install



In [6]:
//export
public func ll(){
  print("/bin/ls".shell("-lh"))
}

In [7]:
ll()

total 92K
-rw-rw-r-- 1 ubuntu ubuntu  460 Sep 14 22:52 blank_swift.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  12K Sep 14 22:52 convolutional_neural_network.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 4.0K Sep 15 12:41 deep-learning-box.sh
-rw-rw-r-- 1 ubuntu ubuntu  622 Sep 14 22:57 inference.swift
-rw-rw-r-- 1 ubuntu ubuntu  20K Sep 14 22:52 initial_setup.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 6.9K Sep 14 22:52 layers.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  33K Sep 14 22:52 matrix_multiplication.ipynb
drwxrwxr-x 4 ubuntu ubuntu 4.0K Sep 15 12:53 swift-install



In [8]:
print("yeet")

yeet


In [9]:
//export
public func downloadFile(_ url: String, dest: String? = nil, force: Bool = false) {
    let dest_name = dest ?? (Path.cwd/url.split(separator: "/").last!).string
    let url_dest = URL(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if !force && Path(dest_name)!.exists { return }

    print("Downloading \(url)...")

    if let cts = Just.get(url).content {
        do    {try cts.write(to: URL(fileURLWithPath:dest_name))}
        catch {print("Can't write to \(url_dest).\n\(error)")}
    } else {
        print("Can't reach \(url)")
    }
}

In [10]:
//export
import TensorFlow

In [11]:
//export
protocol ConvertibleFromByte: TensorFlowScalar {
    init(_ d:UInt8)
}

In [12]:
//export
extension Float : ConvertibleFromByte {}
extension Int32 : ConvertibleFromByte {}

In [13]:
//export
extension Data {
    func asTensor<T:ConvertibleFromByte>() -> Tensor<T> {
        return Tensor(map(T.init))
    }
}

In [14]:
//export
func loadMNIST<T: ConvertibleFromByte>
            (training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k"
    let kind = labels ? "labels" : "images"
    let batch = training ? 60000 : 10000
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists {
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return data.asTensor() }
    else      { return data.asTensor().reshaped(to: shape)}
}

public func loadMNISTData(path:Path, flat:Bool = false)
        -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    try! path.mkdir(.p)
    return (
        loadMNIST(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: true,  labels: true,  path: path, flat: flat),
        loadMNIST(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: false, labels: true,  path: path, flat: flat)
    )
}

In [15]:
//export
public let mnistPath = Path.home/".machine-learning"/"data"/"mnist_home"

In [16]:
let (xTrain, yTrain, xValid, yValid) = loadMNISTData(path: mnistPath)
xTrain.shape

▿ [60000, 28, 28]
  ▿ dimensions : 3 elements
    - 0 : 60000
    - 1 : 28
    - 2 : 28


In [17]:
//export 
import Dispatch

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ f: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating > 1 { f() }
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        f()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}

In [18]:
time(repeating:10) {
    print("yeet")
}

yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet
yeet
average: 0.0056475 ms,   min: 0.000912 ms,   max: 0.010002 ms


In [26]:
// export
public extension String {
    func findFirst(pat: String) -> Range<String.Index>? {
        return range(of: pat, options: .regularExpression)
    }
    func hasMatch(pat: String) -> Bool {
        return findFirst(pat:pat) != nil
    }
}

In [27]:
//export
public func notebookToScript(fname: Path){
    let newname = fname.basename(dropExtension: true)+".swift"
    let url = fname.parent/"FastaiNotebooks/Sources/FastaiNotebooks"/newname
    do {
        let data = try Data(contentsOf: fname.url)
        let jsonData = try JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED, FAM! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells {
            if let source = cell["source"] as? [String], !source.isEmpty, 
                   source[0].hasMatch(pat: #"^\s*//\s*export\s*$"#) {
                module.append("\n" + source[1...].joined() + "\n")
            }
        }
        try module.write(to: url, encoding: .utf8)
    } catch {
        print("Can't read the content of \(fname)")
    }
}

In [28]:
// export
public func exportNotebooks(_ path: Path) {
    for entry in try! path.ls()
    where entry.kind == Entry.Kind.file && 
          entry.path.basename().hasMatch(pat: #"^\d*_.*ipynb$"#) {
        print("Converting \(entry)")
        notebookToScript(fname: entry.path)
    }
}

In [29]:
// export
import NotebookExport

public func exportSchwiftyNotebook(notebookName: String){
    let exporter = NotebookExport(Path.cwd/"\(notebookName).ipynb")
    print(exporter.export(usingPrefix: "SchwiftyNotebook_"))
}

In [33]:
// export
public func famRavioli(){
    print("yeet yeet yeet, fam ravioli")
}

In [34]:
exportSchwiftyNotebook(notebookName: "initial_setup")

success
